<a href="https://colab.research.google.com/github/junjy007/UTS_ML2019_Main/blob/master/NB03_NeuralNets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Basic Neural Networks

## 1.1 Prepare Environment

### 1.1.1 Library Import

In [0]:
import numpy as np
import math
import torch
import plotly.express as px
import pandas as pd
import random

### 1.1.2 Visualisation Functions

#### A Simple visualiser

In [0]:
def simple_visualise(mod):
    xx, yy = np.meshgrid(np.arange(-5, 5.01, 0.05), 
                         np.arange(-5, 5.01, 0.05))
    xx = xx.flatten()
    yy = yy.flatten()
    d = pd.DataFrame(data=dict(x=xx, y=yy, 
                               z=[mod.forward((x, y)) 
                                  for x, y in zip(xx, yy)]))
    fig = px.scatter(d, x='x', y='y', color='z')
    fig.show()

## 1.2 Neural Networks Model Definition

### 1.2.1 Definition and Naive Implementation

Let us literally translate the definition of a neural network into computer implementation:
Neural network: Multiple _layers_ of _perceptron(s)_.
```python
def compute_neural_network(x):
    # 0. prepare the input for the first layer
    layer_input = x
    for layer_idx in [0, 1, 2]:
        # 1. fill output of this layer by executing each
        #    perceptron in this layer
        layer_output = []
        for perceptron in net_layers[layer_idx]:
            perceptron.compute_output(layer_input) 
            # Note all perceptrons in this layer share the same
            # `layer_input`
        #!!----------------------------------    
        # 2. pass the output of THIS layer
        #    to the NEXT layer as the input
        #------------------------------------
        layer_input = layer_output
        # END OF LOOP OVER `layer_idx`
```
Recall that a perception is to get the weighted sum of all attributes in an input, followed by some _activation_. See below:
```python
def compute_perceptron(x):
    weighted_sum = sum([xi * wi for xi, wi in zip(x, weights)])
    return activation_function(weighted_sum)
```
Of course, we will need to get the weights and the activation function setup. So we will use an object class to represent both the perceptrons and the networks.

NB: If you don't understand the construction `[t for t in list_of_t_values]`, please checkout tutorials about Python list.

In [0]:
def sigmoid_func(a):
    return 1 / (1 + math.exp(-a))



class Perceptron2D:
    """Perceptron model: linearly combine data attributes followed by a non-linear activation
    This is a simplified implementation and deals with data with 2 attributes. 
    You can also refer to the more complete implementation in the note of Week 3.
    """
    def __init__(self, w0=1, w1=0, activation_func=sigmoid_func):
        self.w0 = w0
        self.w1 = w1
        self.act = activation_func
    
    def forward(self, x):
        wsum = x[0] * self.w0 + x[1] * self.w1
        sigmoid_wsum = self.act(wsum) # sigmoid for activation
        return sigmoid_wsum
        

In [0]:
# Let's have a look at how the perceptron worked on 2D data
p = Perceptron2D(0.1, -0.5)
simple_visualise(p)
# Please note the effect of activation by examining the z-value.

__EXERCISE__

In the code cell above, adjust the model parameters and observe the change of the model behaviour.

__EXERCISE__

Use a different activation function. Such as
$$
\begin{align}
y(h) = \left\{ \begin{array}{c}
0, \textrm{ if } h \leq 0 \\
h, \textrm{ if } h > 0
\end{array} \right.
\end{align}
$$

Add implement your activation like:
```python
def relu_func(h):
    # compute y
    # HINT: consider using `max`
    return y
```
Then use your function to construct a Perceptron check the behaviour of the perceptron.

In [0]:
def my_activation_func(h):
    return h
p = Perceptron2D(0.1, -0.5, activation_func=my_activation_func)
simple_visualise(p)

In [0]:
def k2(x):
    return [xi**2 for xi in x]

class Perceptron2DX:
    """Perceptron model wrapped. We transform the input before
    processing them using the perceptron.
    """
    def __init__(self, percep, xtransform=k2):
        self.perceptron = percep
        self.xtransform = xtransform
    
    def forward(self, x):
        return self.perceptron.forward(self.xtransform(x))

In [0]:
pp = Perceptron2DX(p)
simple_visualise(pp)

__EXERCISE__

__1__:
Use a different transform function. Such as
$$
\begin{align}
x'_1 = \sin(\omega_1 \cdot x_1) \\
x'_2 = \cos(\omega_2 \cdot x_2)
\end{align}
$$

__2__:
Using your transform function on a _different perceptron core_, e.g.
```python
vanilla_perceptron_2 = Perceptron2D(
        -1, 1, 
        activation_func=my_activation_func)
pp2a = Perceptron2DX(vanilla_perceptron_2, 
                     xtransform=new_transform)
```


In [0]:
def new_transform(x):
    tx0 = math.sin(x[0] * 5)
    tx1 = math.cos(x[1])
    return [tx0, tx1]

pp2 = Perceptron2DX(p, xtransform=new_transform)
simple_visualise(pp2)

In [0]:
ly0_p0 = Perceptron2D(+1, -1, activation_func=my_activation_func)
ly0_p1 = Perceptron2D(-1, -3, activation_func=my_activation_func)
ly1_p = Perceptron2D(-1, 0.5, activation_func=math.tanh)
def new_transform(x):
    tx0 = ly0_p0.forward(x)
    tx1 = ly0_p1.forward(x)
    return [tx0, tx1]

pp3 = Perceptron2DX(ly1_p, xtransform=new_transform)
simple_visualise(pp3)

In [0]:
# Try yourself: can you change the parameters so 
ly0_p0 = Perceptron2D(+1, -1, activation_func=lambda x:x)
ly0_p1 = Perceptron2D(-1, -3, activation_func=lambda x:x)
ly1_p = Perceptron2D(-1, 0.5, activation_func=math.tanh)
def new_transform(x):
    tx0 = ly0_p0.forward(x)
    tx1 = ly0_p1.forward(x)
    return [tx0, tx1]

pp3 = Perceptron2DX(ly1_p, xtransform=new_transform)
simple_visualise(pp3)

## 1.2.2 Multiple Layer Perceptron

Alternatively (to the nested perceptrons above), we can define an NeuralNet class to hold all perceptrons in all layers. The advantage is that now we can easily extend the network to have more layers. 

#### Naive Implementation

In [0]:
# Define a class, so our network can manage its "perceptrons" easily
class NeuralNet:
    """NeuralNet represents a simple neural network object class.
    As an example, it consists of 2 layers of perceptrons. 
    The first layer has 2 perceptrons and the second one has 1.
    
    The perceptrons deal with data of 2 attributes.
    """
    def __init__(self, perc0_w=(-1, 1), perc1_w=(2, -1), perc2_w=(0.5, 0.5)):
        self.layers = [[Perceptron2D(*perc0_w), Perceptron2D(*perc1_w)], 
                       [Perceptron2D(*perc2_w)]] # *(w0,w1) expand the values in tuple
        
    def forward(self, x):
        """
        Compute the network output layer by layer. "forward" is a conventional
        term for execute computing of a net.
        """
        # use layer-0 to process x and get what's to feed to layer-1
        layer1_input = [p.forward(x) for p in self.layers[0]]
        # get the final output from layer-1
        final_output = [p.forward(layer1_input) for p in self.layers[1]]
        # note we have only two layers, so I didn't use a loop over the layers
        return final_output[0]
        

In [0]:
net0 = NeuralNet()
simple_visualise(net0)

__EXERCISE__ (optional, similar to one above)

Adjust parameters to show how the network behaviour changes.

#### Computation using Matrix Operations

In [0]:
def matmul(A, B):
    """
    matmul computs A x B for two matrices
    :param A: a collective object contains rows of a matrix
        A[i], i-th row, another collective object contains the elements
        A[i][j], the element
    :param B: similar to A
    """
    # figure out the size of A and B and the result
    rows = len(A)
    elements_inner = len(A[0])
    assert elements_inner == len(B), "Rows of B must be the same as Cols of A"
    cols = len(B[0])
    # Initialise C to the appropriate size
    C = [[0.0 for ci in range(cols)] for ri in range(rows)]
    
    # Fill C: 2 outer loops are for each element
    for r in range(rows):
        for c in range(cols):
            # Compute element [i][j]
            for k in range(elements_inner):
                C[r][c] += A[r][k] * B[k][c]
    return C

HINT: read the code and try it while watching the accompany video.

In [0]:
# Define a class, so our network can manage its "perceptrons" easily
class NeuralNetV2:
    """NeuralNetV2 represents a simple neural network object class.
    This object will manage all the neurons in the network. 
    """
    def __init__(self, neuron_numbers_in_layers=[2, 2, 1],
                 weights=None):
        """
        :param neuron_numbers_in_layers: first/last -- inputs and outputs
        :param weights: a dict, weights["in0out1"] represents the weights
          for computing layer1 from layer0, if layer0 has 3 inputs and layer1
          has 2 outputs, then the weight will be a matrix of 3 x 2, i.e.
          weights["in0out1"][i][j] is the weight for computing element-j in layer1
          by using element-i in layer0.
        """
        # Weights between 
        # Layer1 and 2, Layer 2 and 3, ...
        
        self.weights = dict()
        self.activ_fn = dict()
        self.neuron_nums = neuron_numbers_in_layers
        self.layer_num = len(neuron_numbers_in_layers) - 1
        
        # DEFINE (!not DO!, we dont have input X now) computation between layers
        for l_in in range(self.layer_num):
            l_out = l_in + 1
            pkey = f"in{l_in}out{l_out}"
            try:
                # try to use provided weight
                W = weights[pkey] # NOTE: should copy
            except:
                # if not provided ...
                n_in = neuron_numbers_in_layers[l_in]
                n_out = neuron_numbers_in_layers[l_out]
                W = [[random.gauss(0, 0.1) for j in range(n_out)] 
                     for i in range(n_in)] # see init above
            self.weights[pkey] = W
            self.activ_fn[pkey] = math.tanh # you may want to try your own

        
    def forward(self, x):
        """
        DO computations:
        
        Compute the network output layer by layer. "forward" is a conventional
        term for execute computing of a net.
        :param x: a list of list: x[i], sample-i, having a number of input attributes
          if you have only one sample, input it as 
              [[0, 1, 0]], 
              NOT [0, 1, 0]
        """
        layer_input = x
        for l_in in range(self.layer_num):
            # Use layer-in to process x and get what's to feed to layer-out
            # Setup 
            l_out = l_in + 1
            pkey = f"in{l_in}out{l_out}"
            # Compute the weighted sum 
            layer_pre_activation = matmul(layer_input, self.weights[pkey])
            # Perform activation(the construction below is equivalent to nested loops)
            layer_out = list(map(lambda x_:list(map(self.activ_fn[pkey], x_)), 
                            layer_pre_activation))
            layer_input = layer_out # feed the output of this layer to the next layer
        return layer_out
    
    
class VisWrap:
    "Wrap I/O for the new network object for visualisation"
    def __init__(self, nn):
        self.nn = nn
        
    def forward(self, x):
        return self.nn.forward([x])[0][0]

In [0]:
nn2 = NeuralNetV2()
nn2_wrap = VisWrap(nn2)
simple_visualise(nn2_wrap)

In [0]:
# Let's try to adjust the weights. Carefully keep the number of weights 
# consistent with the number of neurons you had set to the layers.
nn2 = NeuralNetV2(
    neuron_numbers_in_layers=[2, 3, 1],
    weights={"in0out1":[[0, 0.5, 1], [-1, -5, 2]],
             "in1out2":[[-1], [+1], [0.5]]})
nn2.activ_fn["in1out2"] = lambda x:max(0, x)
nn2_wrap = VisWrap(nn2)
simple_visualise(nn2_wrap)

## 1.3 Training Neural Nets via Backprop

It is not trivial to come up with a simple rule to adjust all the parameters in the entire neural network stucture (recall when we consider a single perceptron, we did propose intuitive scheme to improve the fitness of the model to data). 

The idea is to take a divide-and-conquer scheme. Let's take a careful look at the computation in one perceptron. Throw the machine learning terminology in wind and focus on the computation steps only.
$$
\begin{align}
a &\leftarrow w_0 \cdot x_0 + w_1 \cdot x_1 \\
h &\leftarrow g(a) \\
Loss &\leftarrow Compare(h, y)
\end{align}
$$

Let us think about the statement "to make the loss smaller" with a bit care: which specific means we could possibily take to "make" the final loss change? During training, we change the model parameters, including $w_{i,j}$. So we need to know the influence on the final loss of each model parameter. In this section we will examine an example of so-called "backpropagation" process.

### 1.3.1 Adjust Parameters to Modify Model Behaviour

Given training data $\{(x_1, y_1), (x_2, y_2), \dots\}$, we would like the net to predict for each $x_i$ the target value $y_i$. If this is the case, then our mission has completed. Of course, this is generally NOT the case if we start from a random set of model parameters.

For example, if we have one training sample $(x=(2.5, 2), y=1.0)$, let us compare the prediction given by the network above and the target value $y=1.0$: 

In [0]:
# First, check the current output of the net
def sigm(x):
    return 1/(1+math.exp(-x))

nn2 = NeuralNetV2(
    neuron_numbers_in_layers=[2, 3, 1],
    weights={"in0out1":[[0, 0.5, 1], [-1, -5, 2]],
             "in1out2":[[-1], [+1], [0.5]]})
nn2.activ_fn["in0out1"] = sigm
nn2.activ_fn["in1out2"] = sigm

nn2_wrap = VisWrap(nn2)
simple_visualise(nn2_wrap)

Let's check the nets behaviour at one data:

In [0]:
nn2.forward([(2.5, 2)])

This is smaller than the desired output 1.0. So we want the output to increase at this $x$. Let's learn how to adjust the network using gradients computed through backprop.

### 1.3.2 Manual Backprop Through Layers

Let us implement the backpropagation for a three layer simple net. 

#### Helper Functions

In [0]:
##############################################################
# HELPER FUNCTIONS
# You do NOT need to learn those to USE modern neural networks
# Those functions provide basic array functions in LOW efficiency
# but clear manner. You may want to check them if you want to
# UNDERSTAND the technical details of NN.
# --------
# First define sigmoid gradient
import math
def sigm(x): # redefine here for reference
    return 1/(1+math.exp(-x))

def gsigmoid(x):
    return math.exp(-x)/(1+math.exp(-x))**2

def elementwise_apply(f, nested_list):
    return list(map(lambda x_:list(map(f, x_)), nested_list))

def elementwise_times(nested_list1, nested_list2):
    return [[a * b for a, b in zip(r1, r2)] 
            for r1, r2 in zip(nested_list1, nested_list2)]

def mat_tr(nested_list):
    return [c for c in zip(*nested_list)]

def shape(nested_list):
    return len(nested_list), len(nested_list[0])
##############################################################

In [0]:
# UNIT Test: gsigmoid -- Test the others.
test_x = [-3, -1, 0, 1, 3, 5]
test_eps = 1e-4
for x_ in test_x:
    numerical_diff = (sigm(x_ + test_eps) - sigm(x_)) / test_eps
    analytic_diff = gsigmoid(x_)
    print(f"NumDiff: {numerical_diff:.3f} ~ AnaDiff: {analytic_diff:.3f}")

#### Backprop

Below I explicily write out the forward method followed by a backward computation.

In [0]:
def special_forward(nn, x):
    """
    This is a special version for manual implementing and testing the backpropagation algorithm. 
    We only use the network 
    We don't use the computation and activation of network `nn` 
    """
    
    # Copy-and-paste and simplify forward computation here:
    layer1_input = x
    layer1_pre_activation = matmul(layer1_input, nn.weights["in0out1"])
    layer1_out = elementwise_apply(sigm, layer1_pre_activation)
    
    layer2_input = layer1_out # feed the output of this layer to the next layer
    
    layer2_pre_activation = matmul(layer2_input, nn.weights["in1out2"])
    layer2_out = elementwise_apply(sigm, layer2_pre_activation)

    layer2_pre_activation_g = elementwise_apply(gsigmoid, layer2_pre_activation)
    w12_g = matmul(mat_tr(layer2_input), layer2_pre_activation_g)
    layer1_out_g = matmul(layer2_pre_activation_g, mat_tr(nn.weights["in1out2"]))
    layer1_pre_activation_g = elementwise_times(
        elementwise_apply(gsigmoid, layer1_pre_activation),
        layer1_out_g)
    w01_g = matmul(mat_tr(layer1_input), layer1_pre_activation_g)

    return layer2_out, w01_g, w12_g
    
# TODO: Mark this somewhere else: this Les Mis is FUNNY! https://www.youtube.com/watch?v=dF495ERjRUo

In [0]:
out, w01_g, w12_g = special_forward(nn2, [[2.5, 2]])

#### Numerical verification

Next, let us check element by element how does our backprop work. The plan is to adjust each adjustable parameter a bit and check the change of the final output.

In [0]:
# test adjusting weights in0out1
wkey = "in0out1"
w_rows, w_cols = shape(nn2.weights[wkey])
numerical_g = [[0 for c in range(w_cols)] for r in range(w_rows)]
test_eps = 1e-4
test_x = [[2.5, 2]]
old_out = nn2.forward(test_x)

for r in range(w_rows):
    for c in range(w_cols):
        old_value = nn2.weights[wkey][r][c] # save the old value to put back after test        
        nn2.weights[wkey][r][c] += test_eps
        new_out = nn2.forward(test_x)
        diff = new_out[0][0] - old_out[0][0] # check the effect of adjusting the corresponding parameter
        numerical_g[r][c] = diff / test_eps
        # put the old value back
        nn2.weights[wkey][r][c] = old_value
        

In [0]:
print("numerical differential")
print(numerical_g)
print("analytical differential")
print(w01_g)

__EXECISE__ [Optional]

1. Read the code and Explain what you had observed.
2. Check the computation for weights transforms data from layer 1 to 2.
3. Integrate the backpropagation function into the network class.



## 1.3.3 Using Computational Framework

Modern framework allows us to easily perform all the steps above. The example above can be reformulated as

In [0]:
import torch
import torch.nn as nn

In [0]:
class MyNN(nn.Module):
    def __init__(self, neuron_numbers_in_layers=[2, 3, 1]):
        super(MyNN, self).__init__()
        
        self.layers = nn.ModuleList(
            [nn.Linear(in_features=nin, out_features=nout)
             for nin, nout in zip(neuron_numbers_in_layers[:-1], 
                                  neuron_numbers_in_layers[1:])])
        
    def forward(self, x):
        h = x
        for l in self.layers:
            h = torch.tanh(l(h))
        return h
    
class TorchVisWrap:
    def __init__(self, nn):
        self.nn = nn
    def forward(self, x):
        y = self.nn(torch.Tensor(x).unsqueeze(dim=0))
        return y.item()
        

In [0]:
torch.manual_seed(42)
nn3 = MyNN([2, 6, 1])
nn3_wrap = TorchVisWrap(nn3)
simple_visualise(nn3_wrap)

Let us perform training, call it changing a neural network behaviour or searching in the hypotheses space of neural networks, it is up to your viewpoint. Eg. We want the net to generate
    + for (4, -4)
    - for (4, 4)
    + for (-4, 4)
    - for (-4, -4)


In [0]:
from torch.optim import Adam
optim = Adam(nn3.parameters(), lr=0.01) # manager: adjust params according to grads

In [0]:
train_steps = 50
visualise_every_n_steps = 10
trn_X = torch.Tensor([[4, -4], [4, 4], [-4, 4], [-4, -4]])
trn_y = torch.Tensor([[1.0], [-1.0], [1.0], [-1.0]])
for it in range(train_steps):
    loss = ((trn_y - nn3(trn_X))**2).sum()
    optim.zero_grad() # reset gradients (to clear computed gradients from previous steps)
    loss.backward() # In one stroke, all gradients are computed!
    optim.step()  # apply the gradients to the parameters
    if it % visualise_every_n_steps == 0:
        # Check the effect
        simple_visualise(nn3_wrap)